# Data Preprocessing
## Wildfire-Induced Power Outages: A Data Mining Analysis

This notebook handles loading, cleaning, and preprocessing all datasets for the California wildfire-power outage analysis.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

# Define paths
RAW_DATA_PATH = '../data/raw/'
PROCESSED_DATA_PATH = '../data/processed/'

print('Libraries imported successfully!')

Libraries imported successfully!


---
## 1. Power Outage Data
### 1.1 Purdue Power Outages Dataset (2000-2016)

In [2]:
# Load Purdue Power Outages Dataset
purdue_outages = pd.read_excel(
    RAW_DATA_PATH + 'purdue_power_outages_2000_2016.xlsx',
    skiprows=5
)

print(f'Purdue Outages Shape: {purdue_outages.shape}')
purdue_outages.head()

Purdue Outages Shape: (1535, 57)


,variables,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,...,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
0,Units,NaN,NaN,NaN,NaN,NaN,NaN,NaN,numeric,NaN,"Day of the week, Month Day, Year",Hour:Minute:Second (AM / PM),"Day of the week, Month Day, Year",Hour:Minute:Second (AM / PM),NaN,NaN,NaN,mins,Megawatt,NaN,cents / kilowatt-hour,cents / kilowatt-hour,cents / kilowatt-hour,cents / kilowatt-hour,Megawatt-hour,...,NaN,NaN,NaN,%,%,%,USD,USD,fraction,%,USD,USD,%,%,NaN,%,%,persons per square mile,persons per square mile,persons per square mile,%,%,%,%,%
1,NaN,1.0,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01 00:00:00,17:00:00,2011-07-03 00:00:00,20:00:00,severe weather,NaN,NaN,3060,NaN,70000.0,11.6,9.18,6.81,9.28,2332915,...,276286.0,10673.0,2595696.0,88.944776,10.644005,0.411181,51268,47586,1.077376,1.6,4802,274182,1.751391,2.2,5348119.0,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
2,NaN,2.0,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11 00:00:00,18:38:00,2014-05-11 00:00:00,18:39:00,intentional attack,vandalism,NaN,1,NaN,NaN,12.12,9.71,6.49,9.28,1586986,...,284978.0,9898.0,2640737.0,88.833534,10.791609,0.37482,53499,49091,1.089792,1.9,5226,291955,1.790002,2.2,5457125.0,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
3,NaN,3.0,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26 00:00:00,20:00:00,2010-10-28 00:00:00,22:00:00,severe weather,heavy wind,NaN,3000,NaN,70000.0,10.87,8.19,6.07,8.15,1467293,...,276463.0,10150.0,2586905.0,88.920583,10.687018,0.392361,50447,47287,1.066826,2.7,4571,267895,1.706266,2.1,5310903.0,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
4,NaN,4.0,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19 00:00:00,04:30:00,2012-06-20 00:00:00,23:00:00,severe weather,thunderstorm,NaN,2550,NaN,68200.0,11.79,9.25,6.71,9.19,1851519,...,278466.0,11010.0,2606813.0,88.895368,10.682239,0.422355,51598,48156,1.071476,0.6,5364,277627,1.932089,2.2,5380443.0,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743


In [3]:
# Filter for California only
ca_purdue_outages = purdue_outages[purdue_outages['U.S._STATE'] == 'California'].copy()

# Drop the units row if present
ca_purdue_outages = ca_purdue_outages[ca_purdue_outages['YEAR'].notna()]
ca_purdue_outages = ca_purdue_outages[ca_purdue_outages['YEAR'] != 'Units']

print(f'California outages: {len(ca_purdue_outages)} records')

California outages: 210 records


In [4]:
# Select relevant columns
purdue_cols = [
    'YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION', 'CLIMATE.REGION',
    'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME',
    'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED'
]

ca_purdue_outages = ca_purdue_outages[[col for col in purdue_cols if col in ca_purdue_outages.columns]].copy()

# Convert data types
ca_purdue_outages['YEAR'] = pd.to_numeric(ca_purdue_outages['YEAR'], errors='coerce').astype('Int64')
ca_purdue_outages['MONTH'] = pd.to_numeric(ca_purdue_outages['MONTH'], errors='coerce').astype('Int64')
ca_purdue_outages['OUTAGE.DURATION'] = pd.to_numeric(ca_purdue_outages['OUTAGE.DURATION'], errors='coerce')
ca_purdue_outages['DEMAND.LOSS.MW'] = pd.to_numeric(ca_purdue_outages['DEMAND.LOSS.MW'], errors='coerce')
ca_purdue_outages['CUSTOMERS.AFFECTED'] = pd.to_numeric(ca_purdue_outages['CUSTOMERS.AFFECTED'], errors='coerce')

# Convert dates
ca_purdue_outages['OUTAGE.START.DATE'] = pd.to_datetime(ca_purdue_outages['OUTAGE.START.DATE'], errors='coerce')
ca_purdue_outages['OUTAGE.RESTORATION.DATE'] = pd.to_datetime(ca_purdue_outages['OUTAGE.RESTORATION.DATE'], errors='coerce')

ca_purdue_outages.info()

<class 'pandas.core.frame.DataFrame'>
Index: 210 entries, 1076 to 1285
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   YEAR                     210 non-null    Int64         
 1   MONTH                    210 non-null    Int64         
 2   U.S._STATE               210 non-null    object        
 3   POSTAL.CODE              210 non-null    object        
 4   NERC.REGION              210 non-null    object        
 5   CLIMATE.REGION           210 non-null    object        
 6   OUTAGE.START.DATE        210 non-null    datetime64[ns]
 7   OUTAGE.START.TIME        210 non-null    object        
 8   OUTAGE.RESTORATION.DATE  198 non-null    datetime64[ns]
 9   OUTAGE.RESTORATION.TIME  198 non-null    object        
 10  CAUSE.CATEGORY           210 non-null    object        
 11  CAUSE.CATEGORY.DETAIL    110 non-null    object        
 12  OUTAGE.DURATION          198 non-null

In [5]:
# Summary
print('California Power Outages Summary:')
print(f"Date Range: {ca_purdue_outages['YEAR'].min()} - {ca_purdue_outages['YEAR'].max()}")
print(f"\nCause Categories:\n{ca_purdue_outages['CAUSE.CATEGORY'].value_counts()}")

California Power Outages Summary:
Date Range: 2000 - 2016

Cause Categories:
CAUSE.CATEGORY
severe weather                   70
system operability disruption    41
islanding                        28
intentional attack               24
equipment failure                21
fuel supply emergency            17
public appeal                     9
Name: count, dtype: int64


### 1.2 DOE Grid Disruptions (2000-2014)

In [6]:
# Load DOE Grid Disruptions
doe_disruptions = pd.read_csv(RAW_DATA_PATH + 'doe_grid_disruptions_2000_2014.csv')

# Filter for California
ca_doe_disruptions = doe_disruptions[
    doe_disruptions['Geographic Areas'].str.contains('California|CA', case=False, na=False)
].copy()

# Convert dates
ca_doe_disruptions['Date Event Began'] = pd.to_datetime(ca_doe_disruptions['Date Event Began'], errors='coerce')
ca_doe_disruptions['Date of Restoration'] = pd.to_datetime(ca_doe_disruptions['Date of Restoration'], errors='coerce')

print(f'California DOE disruptions: {len(ca_doe_disruptions)} records')
ca_doe_disruptions.head()

California DOE disruptions: 300 records


,Event Description,Year,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Respondent,Geographic Areas,NERC Region,Demand Loss (MW),Number of Customers Affected,Tags
13,Electrical System Islanding,2014,2014-06-09,11:07 AM,2014-06-09,11:30 AM,Peak Reliability,"Alberta, Canada",WECC,Unknown,Unknown,islanding
18,Electrical System Islanding,2014,2014-06-03,3:32 PM,2014-06-03,3:59 PM,Peak Reliability,"Alberta, Canada",WECC,338,NaN,islanding
21,Electrical System Separation (Islanding),2014,2014-05-26,12:31 PM,2014-05-26,1:18 PM,Peak Reliability,"British Columbia & Alberta, Canada",WECC,Unknown,Unknown,islanding
22,Physical Attack - Vandalism,2014,2014-05-23,3:00 PM,2014-05-25,7:00 PM,Duke Energy Progress,North Carolina,SERC,Unknown,Unknown,"vandalism, physical"
24,Public Appeal to Reduce Electricity Usage - Wi...,2014,2014-05-16,10:43 AM,2014-05-16,9:00 PM,San Diego Gas & Electric Co,"San Diego & Orange Counties, California",WECC,"3,900","1,400,000","wild fire, public appeal"


### 1.3 US Electric Grid New Dataset (2023)

In [7]:
# Load US Electric Grid New Dataset (2023)
us_grid_new = pd.read_csv(RAW_DATA_PATH + 'US Electric Grid new.csv')

print(f'US Electric Grid New Shape: {us_grid_new.shape}')
print(f'Columns: {us_grid_new.columns.tolist()}')
us_grid_new.head()

US Electric Grid New Shape: (167, 7)
Columns: ['Event Year', 'Event Month', 'Date Event Began', 'Area Affected', 'NERC Region', 'Demand Loss (MW)', 'Number of Customers Affected']


,Event Year,Event Month,Date Event Began,Area Affected,NERC Region,Demand Loss (MW),Number of Customers Affected
0,2023,January,01/01/2023,California: Riverside County;,WECC,0,0
1,2023,January,01/01/2023,California: Sacramento County;,WECC,0,0
2,2023,January,01/01/2023,Florida: Citrus County;,SERC,19,3509
3,2023,January,01/02/2023,Texas: Dallas County;,TRE,0,0
4,2023,January,01/02/2023,New Mexico:,WECC,432,0


In [8]:
# Filter for California records
ca_grid_new = us_grid_new[
    us_grid_new['Area Affected'].str.contains('California', case=False, na=False)
].copy()

# Clean and convert data types
ca_grid_new['Date Event Began'] = pd.to_datetime(ca_grid_new['Date Event Began'], errors='coerce')
ca_grid_new['Event Year'] = pd.to_numeric(ca_grid_new['Event Year'], errors='coerce').astype('Int64')

# Handle 'Unknown' values in numeric columns
ca_grid_new['Demand Loss (MW)'] = pd.to_numeric(
    ca_grid_new['Demand Loss (MW)'].replace('Unknown', np.nan), 
    errors='coerce'
)
ca_grid_new['Number of Customers Affected'] = pd.to_numeric(
    ca_grid_new['Number of Customers Affected'].replace('Unknown', np.nan), 
    errors='coerce'
)

print(f'California records (2023): {len(ca_grid_new)} records')
print(f"\nDate Range: {ca_grid_new['Date Event Began'].min()} to {ca_grid_new['Date Event Began'].max()}")
ca_grid_new.info()

California records (2023): 15 records

Date Range: 2023-01-01 00:00:00 to 2023-06-27 00:00:00
<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, 0 to 159
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Event Year                    15 non-null     Int64         
 1   Event Month                   15 non-null     object        
 2   Date Event Began              15 non-null     datetime64[ns]
 3   Area Affected                 15 non-null     object        
 4   NERC Region                   15 non-null     object        
 5   Demand Loss (MW)              11 non-null     float64       
 6   Number of Customers Affected  12 non-null     float64       
dtypes: Int64(1), datetime64[ns](1), float64(2), object(3)
memory usage: 975.0+ bytes


In [9]:
# Display California 2023 outages
print('California Power Outages (2023):')
ca_grid_new[['Event Year', 'Event Month', 'Date Event Began', 'Area Affected', 
             'NERC Region', 'Demand Loss (MW)', 'Number of Customers Affected']]

California Power Outages (2023):


,Event Year,Event Month,Date Event Began,Area Affected,NERC Region,Demand Loss (MW),Number of Customers Affected
0,2023,January,2023-01-01,California: Riverside County;,WECC,0.0,0.0
1,2023,January,2023-01-01,California: Sacramento County;,WECC,0.0,0.0
10,2023,January,2023-01-07,California: Sacramento County;,WECC,458.0,185434.0
13,2023,January,2023-01-12,California: Sacramento County;,WECC,0.0,0.0
20,2023,January,2023-01-23,California: Sacramento County;,WECC,NaN,NaN
21,2023,January,2023-01-23,California: Sacramento County;,WECC,NaN,NaN
47,2023,February,2023-02-13,California: San Diego County;,WECC,0.0,0.0
51,2023,February,2023-02-19,California: Riverside County;,WECC,0.0,0.0
61,2023,February,2023-02-25,California: Los Angeles County;,WECC,NaN,153555.0
97,2023,March,2023-03-26,Washington: Oregon: Josephine County; Californ...,WECC,34.0,NaN


---
## 2. Wildfire Data
### 2.1 CAL FIRE Incidents (2013-2020)

In [10]:
# Load CAL FIRE Incidents
calfire = pd.read_csv(RAW_DATA_PATH + 'calfire_incidents_2013_2020.csv')

print(f'CAL FIRE Shape: {calfire.shape}')
calfire.head()

CAL FIRE Shape: (1636, 40)


,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Fatalities,Featured,Final,FuelType,Helicopters,Injuries,Latitude,Location,Longitude,MajorIncident,Name,PercentContained,PersonnelInvolved,Public,SearchDescription,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,55,NaN,NaN,NaN,2013-09-06T18:30:00Z,NaN,False,True,NaN,NaN,NaN,37.857000,3 miles east of Groveland along Hwy 120,-120.086000,False,Rim Fire,100.0,NaN,True,The Rim Fire was east of Groveland along Highw...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,19,NaN,NaN,NaN,2013-06-08T18:30:00Z,NaN,False,True,NaN,NaN,NaN,34.585595,Angeles National Forest,-118.423176,False,Powerhouse Fire,100.0,NaN,True,The Powerhouse Fire burned in May and June 201...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,33,NaN,NaN,NaN,2013-07-30T18:00:00Z,NaN,False,True,NaN,NaN,NaN,33.709500,Hwy 243 & Hwy 74 near Mountain Center,-116.728850,False,Mountain Fire,100.0,NaN,True,The Mountain Fire burned in July 2013 off High...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,31,NaN,NaN,NaN,2013-08-30T08:00:00Z,NaN,False,True,NaN,NaN,NaN,39.120000,"Deadwood Ridge, northeast of Foresthill",-120.650000,False,American Fire,100.0,NaN,True,The American Fire burned in August 2013 off De...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10T16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,56,47.0,8.0,117.0,2013-05-11T06:30:00Z,NaN,False,True,NaN,11.0,10.0,0.000000,Southbound Highway 101 at Camarillo Springs Ro...,0.000000,True,Springs Fire,100.0,2167.0,True,"The Springs Fire burned in May 2013, Southboun...","Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02T07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [11]:
# Select relevant columns
calfire_cols = [
    'AcresBurned', 'ArchiveYear', 'Counties', 'Fatalities', 'Injuries',
    'Latitude', 'Longitude', 'Location', 'Name', 'Started', 'Extinguished',
    'StructuresDamaged', 'StructuresDestroyed', 'MajorIncident'
]

calfire_clean = calfire[[col for col in calfire_cols if col in calfire.columns]].copy()

# Convert dates
calfire_clean['Started'] = pd.to_datetime(calfire_clean['Started'], errors='coerce')
calfire_clean['Extinguished'] = pd.to_datetime(calfire_clean['Extinguished'], errors='coerce')

# Calculate fire duration
calfire_clean['Duration_Days'] = (calfire_clean['Extinguished'] - calfire_clean['Started']).dt.days

print(f"Year Range: {calfire_clean['ArchiveYear'].min()} - {calfire_clean['ArchiveYear'].max()}")
print(f"Total Fires: {len(calfire_clean)}")
print(f"Total Acres Burned: {calfire_clean['AcresBurned'].sum():,.0f}")

Year Range: 2013 - 2019
Total Fires: 1636
Total Acres Burned: 7,494,562


### 2.2 US Wildfires SQLite Database (1992-2015)

In [12]:
# Connect to SQLite database
conn = sqlite3.connect(RAW_DATA_PATH + 'us_wildfires_1.88m_fpa_fod.sqlite')

# Query California fires
ca_wildfires_fpa = pd.read_sql_query("""
    SELECT 
        FIRE_YEAR, DISCOVERY_DATE, DISCOVERY_DOY, STAT_CAUSE_DESCR,
        FIRE_SIZE, FIRE_SIZE_CLASS, LATITUDE, LONGITUDE,
        STATE, FIPS_CODE, FIPS_NAME
    FROM Fires
    WHERE STATE = 'CA'
""", conn)
conn.close()

print(f'California Wildfires (FPA FOD): {len(ca_wildfires_fpa):,} records')

California Wildfires (FPA FOD): 189,550 records


In [13]:
# Convert discovery date (Julian date format)
ca_wildfires_fpa['DISCOVERY_DATE'] = pd.to_datetime(
    ca_wildfires_fpa['DISCOVERY_DATE'] - 2440587.5,
    unit='D',
    errors='coerce'
)

print(f"Year Range: {ca_wildfires_fpa['FIRE_YEAR'].min()} - {ca_wildfires_fpa['FIRE_YEAR'].max()}")
print(f"\nFire Causes:\n{ca_wildfires_fpa['STAT_CAUSE_DESCR'].value_counts()}")

Year Range: 1992 - 2015

Fire Causes:
STAT_CAUSE_DESCR
Miscellaneous        51943
Equipment Use        39407
Lightning            27000
Arson                19635
Debris Burning       14318
Missing/Undefined    12605
Campfire              9516
Children              6930
Smoking               5936
Powerline             1198
Railroad               720
Fireworks              219
Structure              123
Name: count, dtype: int64


### 2.3 NASA FIRMS Satellite Data

In [14]:
# Load NASA FIRMS data with FRP
nasa_firms = pd.read_csv(RAW_DATA_PATH + 'nasa_firms_wildfire_data/california_fires_2000_2021_with_frp.csv')

# Convert date
nasa_firms['acq_date'] = pd.to_datetime(nasa_firms['acq_date'], errors='coerce')

print(f'NASA FIRMS: {len(nasa_firms):,} fire detections')
print(f"Date Range: {nasa_firms['acq_date'].min()} to {nasa_firms['acq_date'].max()}")
nasa_firms.head()

NASA FIRMS: 221,183 fire detections
Date Range: 2000-11-01 00:00:00 to 2020-12-31 00:00:00


,latitude,longitude,acq_date,satellite,instrument,frp,type,bright_t31,confidence,year,month
0,32.4646,-114.6906,2011-06-29,Terra,MODIS,91.1,0,315.7,84,2011,6
1,32.4768,-114.6785,2011-10-11,Terra,MODIS,96.2,0,313.7,100,2011,10
2,32.4937,-114.7856,2013-02-06,Terra,MODIS,26.9,0,296.1,65,2013,2
3,32.4987,-114.7985,2010-02-04,Terra,MODIS,22.2,0,282.9,62,2010,2
4,32.5006,-114.7929,2009-07-08,Aqua,MODIS,172.0,0,316.4,78,2009,7


---
## 3. Data Summary

In [15]:
print('=' * 60)
print('DATASET SUMMARY')
print('=' * 60)
print(f"\nPurdue Power Outages (CA): {len(ca_purdue_outages):,} records (2000-2016)")
print(f"DOE Grid Disruptions (CA): {len(ca_doe_disruptions):,} records (2000-2014)")
print(f"US Electric Grid New (CA): {len(ca_grid_new):,} records (2023)")
print(f"CAL FIRE Incidents: {len(calfire_clean):,} records (2013-2020)")
print(f"FPA FOD Wildfires (CA): {len(ca_wildfires_fpa):,} records (1992-2015)")
print(f"NASA FIRMS (CA): {len(nasa_firms):,} records (2000-2021)")
print('\n=> Best overlap for historical analysis: 2000-2015')
print('=> 2023 data available for recent trends')

DATASET SUMMARY

Purdue Power Outages (CA): 210 records (2000-2016)
DOE Grid Disruptions (CA): 300 records (2000-2014)
US Electric Grid New (CA): 15 records (2023)
CAL FIRE Incidents: 1,636 records (2013-2020)
FPA FOD Wildfires (CA): 189,550 records (1992-2015)
NASA FIRMS (CA): 221,183 records (2000-2021)

=> Best overlap for historical analysis: 2000-2015
=> 2023 data available for recent trends


---
## 4. Save Processed Datasets

In [16]:
# Create processed directory
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)

# Save processed datasets
ca_purdue_outages.to_csv(PROCESSED_DATA_PATH + 'california_power_outages.csv', index=False)
ca_doe_disruptions.to_csv(PROCESSED_DATA_PATH + 'california_grid_disruptions.csv', index=False)
ca_grid_new.to_csv(PROCESSED_DATA_PATH + 'california_grid_2023.csv', index=False)
calfire_clean.to_csv(PROCESSED_DATA_PATH + 'california_calfire_incidents.csv', index=False)
ca_wildfires_fpa.to_csv(PROCESSED_DATA_PATH + 'california_wildfires_fpa.csv', index=False)
nasa_firms.to_csv(PROCESSED_DATA_PATH + 'california_nasa_firms.csv', index=False)

print('Processed datasets saved!')
for f in os.listdir(PROCESSED_DATA_PATH):
    if f.endswith('.csv'):
        size = os.path.getsize(PROCESSED_DATA_PATH + f) / (1024*1024)
        print(f'  - {f} ({size:.2f} MB)')

Processed datasets saved!
  - california_nasa_firms.csv (13.52 MB)
  - california_wildfires_fpa.csv (12.52 MB)
  - california_power_outages.csv (0.02 MB)
  - california_grid_disruptions.csv (0.05 MB)
  - california_grid_2023.csv (0.00 MB)
  - california_calfire_incidents.csv (0.26 MB)
